In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import glob

from sklearn.metrics import confusion_matrix
import itertools

from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv1D, MaxPool1D
from keras.initializers import random_uniform
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler as SS

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from matplotlib import pyplot as plt
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding,Dense,LSTM,Dropout,Flatten,BatchNormalization,Conv1D,GlobalMaxPooling1D,MaxPooling1D
from keras.optimizers import  SGD
import matplotlib.pyplot as plt
from keras.regularizers import l2
from keras.optimizers import Adam
from keras import regularizers
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import OneHotEncoder
from keras.preprocessing import sequence
from keras.layers import SimpleRNN
#from hyperas.distributions import uniform

from keras.utils.np_utils import to_categorical
from keras import regularizers
import pandas as pd
import string
import numpy as np
import matplotlib.pyplot as plt

# import warnings
import warnings
# filter warnings
warnings.filterwarnings('ignore')

# Data import

In [ ]:
dataset = pd.read_csv(r'...') # use your path

In [3]:
dataset.head()

dur  proto  service  state  spkts  dpkts  sbytes  dbytes  \
0  0.921987     78        0      5     20      0    1280       0   
1  0.921987     78        0      5     20      0    1280       0   
2  0.921987     78        0      5     20      0    1280       0   
3  0.921987     78        0      5     20      0    1280       0   
4  0.000009     97        0      5      2      0     104       0   

            rate  sttl  ...  ct_dst_sport_ltm  ct_dst_src_ltm  is_ftp_login  \
0      20.607666   254  ...                 1               2             0   
1      20.607666   254  ...                 1               2             0   
2      20.607666   254  ...                 1               2             0   
3      20.607666   254  ...                 1               2             0   
4  111111.107200   254  ...                 1               2             0   

   ct_ftp_cmd  ct_flw_http_mthd  ct_src_ltm  ct_srv_dst  is_sm_ips_ports  \
0           0                 0           1           1                0   
1           0                 0           1           1                0   
2           0                 0           1           1                0   
3           0                 0           1           1                0   
4           0                 0           1           1                0   

   attack_cat  label  
0           7      1  
1           7      1  
2           1      1  
3           2      1  
4           3      1  

[5 rows x 44 columns]

In [4]:
dataset.shape

(329346, 44)

In [6]:
# Splitting dataset into features and labels.
labels = dataset['label']
features = dataset.loc[:, dataset.columns != 'label'].astype('float64')

In [9]:
# For scaling the data, we use RobustScaler class from sklearn.

from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

In [10]:
scaler = RobustScaler()
scaler.fit(features)

features = scaler.transform(features)

In [11]:
from sklearn.preprocessing import LabelEncoder

In [12]:
LE = LabelEncoder()

LE.fit(labels)
labels = LE.transform(labels)

In [13]:
# Labels have been replaced with integers.

np.unique(labels)

array([0, 1], dtype=int64)

In [14]:
# Checking that encoding reversal works.

d = LE.inverse_transform(labels)
d = pd.Series(d)
d.unique()


array([1, 0], dtype=int64)

### Splitting the data

In [15]:
from sklearn.model_selection import train_test_split

In [35]:
# The next step is to split training and testing data. For this we will use sklearn function train_test_split().

features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size= 0.5,random_state=1)

In [36]:
features_train.shape, features_test.shape, labels_train.shape, labels_test.shape

((164673, 43), (164673, 43), (164673,), (164673,))

In [18]:
# this function has evaluation metrics
def report(prediction, y_true):
    print('        ----------Classification Report Of Classes-------------')
    print(classification_report(y_true,prediction))
    print('\n        ----------Validation Data------------------')
    print('Accuarcy:',accuracy_score(y_true, prediction)* 100)
    print('Precision: {:,.4f} %'.format(precision_score(y_true, prediction, average='weighted') * 100))
    print('Recall-score: {:,.4f}'.format(recall_score(y_true, prediction, average='weighted') * 100))
    print('F1-score: {:,.4f}'.format(f1_score(y_true, prediction, average='weighted') * 100))

In [19]:
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

### Logistic Regression

In [20]:
ln = LogisticRegression()

In [21]:
ln.fit(features_train,labels_train)

LogisticRegression()

In [22]:
predict = ln.predict(features_test)
report(predict,labels_test)

        ----------Classification Report Of Classes-------------
              precision    recall  f1-score   support

           0       0.79      0.84      0.82     32881
           1       0.83      0.78      0.81     32989

    accuracy                           0.81     65870
   macro avg       0.81      0.81      0.81     65870
weighted avg       0.81      0.81      0.81     65870


        ----------Validation Data------------------
Accuarcy: 81.1143160771216
Precision: 81.2304 %
Recall-score: 81.1143
F1-score: 81.0980


### GaussianNB

In [23]:
nb = GaussianNB()

In [24]:
nb.fit(features_train,labels_train)

GaussianNB()

In [25]:
predict = nb.predict(features_test)
report(predict,labels_test)

        ----------Classification Report Of Classes-------------
              precision    recall  f1-score   support

           0       0.62      0.94      0.74     32881
           1       0.87      0.42      0.56     32989

    accuracy                           0.68     65870
   macro avg       0.74      0.68      0.65     65870
weighted avg       0.74      0.68      0.65     65870


        ----------Validation Data------------------
Accuarcy: 67.73645058448459
Precision: 74.3741 %
Recall-score: 67.7365
F1-score: 65.4026


### AdaBoostClassifier

In [37]:
ab = AdaBoostClassifier(n_estimators=100)

In [38]:
ab.fit(features_train,labels_train)

AdaBoostClassifier(n_estimators=100)

In [39]:
predict = ab.predict(features_test)
report(predict,labels_test)

        ----------Classification Report Of Classes-------------
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     82053
           1       1.00      1.00      1.00     82620

    accuracy                           1.00    164673
   macro avg       1.00      1.00      1.00    164673
weighted avg       1.00      1.00      1.00    164673


        ----------Validation Data------------------
Accuarcy: 100.0
Precision: 100.0000 %
Recall-score: 100.0000
F1-score: 100.0000


### Decision Tree Classifier

In [49]:
dt = DecisionTreeClassifier()

In [50]:
dt.fit(features_train,labels_train)

DecisionTreeClassifier()

In [51]:
predict = dt.predict(features_test)
report(predict,labels_test)

        ----------Classification Report Of Classes-------------
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     82053
           1       1.00      1.00      1.00     82620

    accuracy                           1.00    164673
   macro avg       1.00      1.00      1.00    164673
weighted avg       1.00      1.00      1.00    164673


        ----------Validation Data------------------
Accuarcy: 100.0
Precision: 100.0000 %
Recall-score: 100.0000
F1-score: 100.0000


### RandomForestClassifier

In [46]:
rf = RandomForestClassifier(n_estimators=100)

In [47]:
rf.fit(features_train,labels_train)

RandomForestClassifier()

In [48]:
predict = rf.predict(features_test)
report(predict,labels_test)

        ----------Classification Report Of Classes-------------
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     82053
           1       1.00      1.00      1.00     82620

    accuracy                           1.00    164673
   macro avg       1.00      1.00      1.00    164673
weighted avg       1.00      1.00      1.00    164673


        ----------Validation Data------------------
Accuarcy: 100.0
Precision: 100.0000 %
Recall-score: 100.0000
F1-score: 100.0000
